<a href="https://colab.research.google.com/github/k-mundinger/public/blob/master/collaborative_filtering/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative filtering

This notebook accompanies the blogpost about **collaborative filtering**. You can find the blogpost here: https://dida.do/blog/collaborative_filtering

The techniques will be illustrated on the famous [MovieLens-100K](https://grouplens.org/datasets/movielens/100k/) dataset. It contains 100k user-movie rating pairs from 943 users on 1682 movies.

We'll use the [surprise](https://surprise.readthedocs.io/en/stable/index.html) library for python which comes with implementations of some prominent collaborative filtering algorithms. We start with importing it along with some other dependencies.

In [ ]:
# only run this cell if you are on colab, otherwise follow the steps described in the readme

!pip install scikit-surprise
!pip install matplotlib
!pip install tqdm
!pip install scikit-learn
!pip install numpy

In [ ]:
from urllib.request import urlretrieve
from collections import Counter
import zipfile
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm.auto import tqdm
from surprise import SVD
from surprise import Dataset, Reader, Prediction
from surprise.prediction_algorithms import BaselineOnly
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
colors = ["#27898B", "#34B3B6", "#83D4D6", "#C583D6" , "#8534B6"]
custom_cmap = ListedColormap(name = "dida_cmap", colors = colors)

We start by downloading the dataset. Note that we _could_ use the built in dataset from the surprise library, however in the next blog post we are going to reuse it so we will need the raw files anyway.

In [ ]:
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()

In [ ]:
# only run this cell if you are on colab

!apt-get install tree

In [ ]:
!tree ml-100k

We have downloaded quite a lot of files. However, the only relevant file for this blog post will be `u.data`, which contains a collection of user-movie ratings in long format. The files `u.user` and `u.item` contain additional information about the users and movies, but since we focus on purely collaborative filtering in this blogpost, they are irrelevant. The dataset comes with predefined splits which one may use for n-fold cross validation, but we will create a split ourselves later.

In [ ]:
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp'] # columns in the dataset
ratings = pd.read_csv(
            'ml-100k/u.data', 
            sep='\t', 
            names=ratings_cols, 
            encoding='latin-1'
                     )

ratings["rating"] = ratings["rating"].apply(lambda x: float(x)) # the ratings are parsed as strings
ratings = ratings.drop("unix_timestamp", axis = 1) # the unix timestamp is irrelevant for us

ratings

So, our dataset consists of in total 100k user-movie-pairs. Even though we don't have meta information about the users and the movies, we can still do some EDA (exploratory data analysis). Let's see how the number of ratings is distributed:

In [ ]:
movie_counts = ratings.groupby("movie_id").count()["rating"]
sorted_movie_counts = sorted(movie_counts, reverse = True)

user_counts = ratings.groupby("user_id").count()["rating"]
sorted_user_counts = sorted(user_counts, reverse = True)

rating_distribution = ratings.groupby("rating").count()["user_id"]



In [ ]:
fig, axs = plt.subplots(ncols = 3, figsize = (30, 8), facecolor="white")

axs[0].plot(sorted_movie_counts, lw = .4, c = "black")
axs[0].set_ylim(0, 520)
axs[0].set_xlim(0, 1682)
axs[0].set_xlabel("Movies sorted by number of ratings they have received")
axs[0].set_ylabel("# Ratings")
axs[0].set_title("Movies")
axs[0].fill_between(np.arange(len(sorted_movie_counts)), np.zeros_like(sorted_movie_counts), sorted_movie_counts, color = "#34B4B6", alpha = .8)

axs[1].plot(sorted_user_counts, lw = .4, c = "black")
axs[1].set_ylim(0, 600)
axs[1].set_xlim(0, 943)
axs[1].set_xlabel("User sorted by number of movies they have rated")
axs[1].set_ylabel("# Ratings")
axs[1].set_title("Users")
axs[1].fill_between(np.arange(len(sorted_user_counts)), np.zeros_like(sorted_user_counts), sorted_user_counts, color = "#34B4B6", alpha = .8)

axs[2].bar(x = rating_distribution.index, height = rating_distribution, color = "#34B3B6", edgecolor = "black")
axs[2].set_title("Rating frequency histogram")


plt.show()

As we can see from these plots, most of the movies have a small number of ratings and likewise most of the users have rated very little items. This is very common in recommendation systems. Notice that there are 1586126 ( = 943 * 1682) possible user movie interactions, however only 100k of those actually have a rating - that's only ~ 6.3 %! This is also known as the _long tail_ phenomenon.  

Another way to phrase is is to say that the user-movie matrix is very _sparse_. Let's have a look at this matrix.

In [ ]:
user_movie_matrix = ratings.pivot_table("rating", index = "user_id", columns = "movie_id")
user_movie_matrix

In [ ]:
plt.figure(figsize = (20, 10))
plt.title("User movie matrix, \n 1 (bad) - 5 (good), white means no rating")

im = plt.matshow(user_movie_matrix, cmap = custom_cmap, fignum = 0) # Fill the missing values with 0s so the image can be rendered properly
plt.colorbar(im, ticks = list(range(1,6)))
plt.xlabel("Users")
plt.ylabel("Movies")

plt.show()

In [ ]:
plt.figure(figsize = (8, 8))
plt.title("Zoom into the user movie matrix, \n 1 (bad) - 5 (good), white means no rating")

plt.matshow(user_movie_matrix.iloc[:50, :50], cmap = custom_cmap, fignum = 0)
plt.colorbar(im, ticks = list(range(1,6)))
plt.xlabel("Users")
plt.ylabel("Movies")

plt.show()

The images confirm the initial observations about the sparsity of this matrix. Another interesting thing to point out, is that there are visible correlations between the columns and the rows of the matrix. This makes sense intuitively, as there are movies which simply are good (and hence receive mainly good ratings) - this explains the similarity between the rows. Also, there are users who tend to give better ratings than others - this explains the correlations between the columns.

In the case of purely collaborative filtering, the task we want to solve now is to predict, how each user would rate each movie - and hence the missing entries in the user-movie matrix. To evaluate the performance of the algorithms, we need to create a hold out test set. Ideally, we would want each user *and* each movie to be present in both the train and the test set. But what necessarily has to hold, is that every user and every movie is still present in the train set after the split - otherwise we could not make predictions about those! This is the well known **cold start problem** from which collaborative filtering suffers severly.

Since some movies in the dataset have only been rated by one user, we need to sort those out, as otherwise the stratified train test split would not be possible.

In [ ]:
movies_rated_at_least_twice = movie_counts[movie_counts >= 2].index.to_list()
filtered_ratings = ratings.loc[ratings.movie_id.isin(movies_rated_at_least_twice)]

In [ ]:
len(filtered_ratings.movie_id.unique())

Unfortunately, that way we lost almost 100 movies. However, for these movies it would have been very difficult to make meaningful predictions anyway, and with the data available we also could not have evaluated these predictions.

In [ ]:
from sklearn.model_selection import train_test_split


def generate_train_test_split(df, test_size, random_state = 31415):

    train_df, test_df = train_test_split(df, test_size = test_size, stratify = df.movie_id, random_state = random_state)

    print(len(train_df.user_id.unique()))
    print(len(test_df.user_id.unique()))

    print(len(train_df.movie_id.unique()))
    print(len(test_df.movie_id.unique()))

    return train_df, test_df

train_df, test_df = generate_train_test_split(filtered_ratings, test_size = .3)

In [ ]:
train_df

To use the surprise library, we now need to create a `Dataset` object. The API of this library takes some getting used to.

In [ ]:
trainset = Dataset.load_from_df(train_df[["user_id", "movie_id", "rating"]], 
                                Reader(rating_scale = (1,5))).build_full_trainset()

testset = Dataset.load_from_df(test_df[["user_id", "movie_id", "rating"]], 
                               Reader(rating_scale=(1,5))).build_full_trainset().build_testset()

To train such a matrix factorization based algorithm, we'll need to split our data into train and test set. We can use the built in method from `surprise`.

Let's have a look at how the data got split.

In [ ]:
test_df["user_id"] = test_df["user_id"].astype(int)
test_df["movie_id"] = test_df["movie_id"].astype(int)

pivoted_train = train_df.pivot_table("rating", index = "user_id", columns = "movie_id")
pivoted_test = test_df.pivot_table("rating", index = "user_id", columns = "movie_id")

In [ ]:
from matplotlib.patches import Patch

train_pairs = (~np.isnan(pivoted_train.values)).astype(int)
test_pairs = (~np.isnan(pivoted_test.values)).astype(int)*2 


vis_split = np.where((train_pairs + test_pairs) > 0, train_pairs + test_pairs, np.nan)

fig, ax = plt.subplots(figsize = (16, 9))

ax.set_title("Visualization of the Train Test Split (only  a small part)")
im = ax.matshow(vis_split[:120, :200], cmap = custom_cmap)

legend = [Patch(facecolor= "#27898B", label = "Train"),
          Patch(facecolor= "#8534B6", label = "Test")
         ]
plt.legend(handles = legend)

plt.show()

## Evaluation and defining a baseline



### Evaluation

So, how are we going to evaluate the predictions? Note first, that most common strategies phrase the prediction of ratings as a _regression_ problem - that means, that the output of our models will be a _continuous_ variable, even though the ratings are discerete (ranging from 1-5 only). For regression problems, typical evaluation metrics are the MAE (mean absolute error) and the MSE (mean squared error), as well as the RMSE (root mean squared error).

### Baseline

To have something to compare our results to later on, let's start with a very simple and unpersonalized baseline - for any unknown user-movie pair, we predict the mean of all ratings that other users have given this film. At the end, when we turn these algortihms into recommendation engines, this would correspond to just recommend the best rated movies. So let's compute the mean ratings for all the movies on our trainset.

In [ ]:
movie_means = {}

for idx in tqdm(train_df.movie_id.unique()):

    movie_means[idx] = train_df.loc[train_df.movie_id == idx].rating.mean()

In [ ]:
test_preds = test_df.movie_id.apply(lambda x: movie_means[x])
print("MAE: ", mean_absolute_error(test_preds, test_df.rating))
print("RMSE:", mean_squared_error(test_preds, test_df.rating, squared = False))
print("R2: ", r2_score(test_df.rating, test_preds))

Let's have a look at how the full user-movie matrix looks like now.

In [ ]:
from surprise import PredictionImpossible

def get_estimate(model, user, item, kind = None):

    if isinstance(model, dict):

        try:
            if kind == "user":
                return model[user]
            else:
                return model[item]
        except:
            KeyError
            return None

    try:
        estimate = model.estimate(user, item)
        if isinstance(estimate, tuple):
            estimate = estimate[0] # inconsitent model outputs in surprise

    except PredictionImpossible:

        estimate = np.nan

    return estimate


def get_full_user_item_matrix(model, users, items):

    full_matrix = [
                            [
                               get_estimate(model, i, j) for j in items
                            ] for i in users
                    ]

    return np.array(full_matrix)

In [ ]:
def show_matrix(full_matrix, title = "Full user movie matrix"):

    fig, ax = plt.subplots(figsize = (16, 9))

    ax.set_title(title)

    im = ax.matshow(full_matrix, cmap = LinearSegmentedColormap.from_list("cont_cmap", colors))
    plt.colorbar(mappable = im)

    plt.show()


def bin_to_integer_ratings(array):

    """
    Rounds the float prediction to the nearest of the integers {1, 2, 3, 4, 5}.
    """ 
    
    return np.digitize(array, [1.5, 2.5, 3.5, 4.5]) + 1

def show_rating_distribution(full_matrix, show_gt = True, model = None, **kwargs):

    fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (24, 8))

    axs[0].set_title("Histogram of all predicted (non-clipped) ratings")
    axs[0].hist(full_matrix.flatten(), bins = 100, color = "#34B3B6")
    axs[0].grid()

    axs[1].set_title("Histogram of the binned ratings")
    flattened_integer_entries = bin_to_integer_ratings(ratings[["user_id", "movie_id"]].apply(lambda x: get_estimate(model, x[0], x[1], **kwargs), axis = 1).values)
    pred_dist = pd.Series(flattened_integer_entries).value_counts()
    axs[1].bar(x = pred_dist.index + .2, height = pred_dist, color = "#34B3B6", width = .4, label = "predicted")


    if show_gt:
        axs[1].bar(x = rating_distribution.index - .2, height = rating_distribution, color = "#8534B6", width = .4, label = "ground truth")
        axs[1].set_title("Histogram of the 100k binned ratings")
    axs[1].legend()
    
    axs[1].grid()

    plt.show()

In [ ]:
full_naive_baseline_matrix = np.stack([np.repeat(movie_means[movie_id], 943) for movie_id in pivoted_train.columns]).transpose()

In [ ]:
show_matrix(full_naive_baseline_matrix, "User-movie-matrix recovered from just predicting the mean for each movie.")

In [ ]:
show_rating_distribution(full_naive_baseline_matrix, model = movie_means)

Note that this approach is definitely not the smartest. We have not considered any information about the users at all. In fact, the rank of this user-movie matrix is one, as all columns are constant!

Just out of curiosity, we may do the same for the users. Note however, that from a recommendation engine perspective this is useless, as by construction for each user all movies will have the same rating.

In [ ]:
user_means = {}

for idx in tqdm(train_df.user_id.unique()):

    user_means[idx] = train_df.loc[train_df.user_id == idx].rating.mean()

test_preds = test_df.user_id.apply(lambda x: user_means[x])

full_naive_user_baseline_matrix = np.stack([np.repeat(user_means[user_id], 1541) for user_id in pivoted_train.index])

In [ ]:
show_matrix(full_naive_user_baseline_matrix, "User-movie-matrix recovered from just predicting the mean for each user.")

In [ ]:
show_rating_distribution(full_naive_user_baseline_matrix, model = user_means, kind = "user")

In [ ]:
test_preds = test_df.user_id.apply(lambda x: user_means[x])
print("MAE :", mean_absolute_error(test_preds, test_df.rating))
print("RMSE:", mean_squared_error(test_preds, test_df.rating, squared = False))
print("R2  :", r2_score(test_df.rating, test_preds))

The algorithm, that is proposed as a baseline from the surprise library, is a bit more sophisticated. For each user *and* each item it introduces a (scalar) _bias_, $b_u$ and $b_i$, respectively. These biases are learnable parameters. The prediction is then defined as $p_{ui} = \mu + b_u + b_i$, where $\mu$ is the mean of _all_ given ratings on the train set. 

In [ ]:
baseline_model = BaselineOnly(verbose = False)

baseline_model.fit(trainset)

In [ ]:
test_preds = baseline_model.test(testset) # surprise requires to construct a testset object first
train_preds = baseline_model.test(trainset.build_testset())

In [ ]:
def get_metrics_from_surprise(pred_object, clip = False, print_ = True):

    _, _, true_ratings, pred_ratings, _ = zip(*pred_object)

    if clip:
        pred_ratings = np.clip(pred_ratings, 1, 5)

    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = mean_squared_error(true_ratings, pred_ratings, squared = False)
    r2 = r2_score(true_ratings, pred_ratings)

    if print_:
        print("MAE   :  ", mae)
        print("RMSE  :  ", rmse)
        print("R2    :  ", r2)

    return mae, rmse, r2

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
get_metrics_from_surprise(train_preds)

In [ ]:
full_baseline_matrix = get_full_user_item_matrix(baseline_model, pivoted_train.index, pivoted_train.columns)

In [ ]:
show_matrix(full_baseline_matrix, "User-movie-matrix as predicted by the baseline model.")

This matrix looks much more diverse than the 2 previous ones and the metrics are better as well. However, the correlations between the rows and the columns are still clearly visible, which makes sense as they reflect the learned biases for each user and item.

In [ ]:
show_rating_distribution(full_baseline_matrix, model = baseline_model)

## Nearest neighbour based algorithms

Let's start with the first "real" algorithm now. I chose to present an algorithm based on kNN (k nearest neighbours), as I think it is the most intuitive one and reflects the underlying idea of collaborative filtering very well. It goes like this: 

For a given user-movie interaction, find the _k_ (this is a hyperparameter of this algorithm) nearest neighbours of the user _who have also rated this movie_. Take the mean of the ratings of the found users for this movie - that is the prediction. 

As we don't have further information about the users in our setting, we characterize each user with the vector of ratings he has given. So each user is identified with a vector $u_i \in \mathbb{R}^{1541}$, as we have 1541 in our filtered dataset. Recall, that for most of the users most of the entries of this vector will be empty. So to search for nearest neighbours, means to find vectors which are _closest_ to the vector of the given user. What closest means, depends on the choice of the _distance_ or _similarity_ measure. Popular choices are the euclidean distance or the cosine distance. For a pair of sparse vectors (which we deal with here), these measures are computed only over the entries which are present in both vectors.

So let's have a look what results this approach gives us!


In [ ]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

knn_basic = KNNBasic(k = 30, 
                     min_k = 1, 
                     verbose = False,
                     sim_options = {"name": "msd"} # msd = mean squared distance = euclidean distance 
                     )

knn_basic.fit(trainset)

test_preds = knn_basic.test(testset)
train_preds = knn_basic.test(trainset.build_testset())

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
get_metrics_from_surprise(train_preds)

Here we chose to take into account the 40 nearest neighbors (if there are that many) and chose the euclidean distance as a similarity measure. The cosine distance produces results which are much worse, this is however readily explained / illustrated at the extreme example, where 2 sers have only one rated movie in common, but one gave it the best rating (5) and the other one the worst rating (1). Since the cosine similarity normalizes the (in this case 1 dimensional) vectors to unit norm, they turn out to be most similar - which is the total opposite of what we want in this case!

In [ ]:
full_knn_matrix = get_full_user_item_matrix(knn_basic, list(range(943)), list(range(1541))) # this can take a while

In [ ]:
show_matrix(full_knn_matrix, "User movie matrix as predicted by the basic kNN model")

In [ ]:
show_matrix(full_knn_matrix[-100:, -100:], "User movie matrix as predicted by the basic kNN model")

In [ ]:
show_rating_distribution(full_knn_matrix, model = knn_basic)

In [ ]:
np.isnan(full_knn_matrix).sum() # number of np.nan entries in this matrix

Notice that, especially in the right part of this matrix, there are many little white dots - in total there are 3460 of those. These correspond to user-item pairs, where the algorithm had too little information to make a prediction. So even though there might have been other users that rated this particular film, they did not have a single _other_ film in common with this user, so there was no way to compute similarities and hence the user did not have neighbours in this space.

This is a major drawback of neighbor based algorithms in such sparse scenarios. An obvious remedy would be to just predict the mean of this movie or the global mean of all ratings in such a case.

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9))

ax.set_title("User-movie-matrix as predicted by the basic kNN model. \n Impossible predictions are filled with the mean over all ratings.")

im = ax.matshow(pd.DataFrame(full_knn_matrix).fillna(trainset.global_mean).values[-100:, -100:], cmap = LinearSegmentedColormap.from_list("cont_cmap", colors))

plt.colorbar(mappable = im)

plt.show()

Because it is very easy with the surprise library, we can quickly try out other variants of the kNN Algorithm. One such Variant takes into account the mean rating for each user when making its prediction.

In [ ]:
knn_means = KNNWithMeans()

knn_means.fit(trainset)

test_preds = knn_means.test(testset)
train_preds = knn_means.test(trainset.build_testset())

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
get_metrics_from_surprise(train_preds)

In [ ]:
full_knn_matrix = get_full_user_item_matrix(knn_means, list(range(943)), list(range(1541))) # this can take a while

In [ ]:
show_matrix(full_knn_matrix, "User-movie-matrix as predicted by the kNN model which corrects the means.")

Notice how suddendly the range of the predictions increased drastically and the model outputs values as high as 7 and as low as -1. This is because the formula involves another addition now and does not only take averages of already existing values, as our previous algorithms did. We can easily fix this by clipping the values at 1 and 5.

In [ ]:
show_matrix(np.clip(full_knn_matrix, 1, 5), "User-movie-matrix as predicted by the kNN model which corrects the means. \n (Predictions are clipped to [1, 5])")

In [ ]:
show_rating_distribution(full_knn_matrix, model = knn_means)

Overall, this variant:
* made our evaluation metrics better
* produced a more diverse user-movie matrix
* got rid of the problem of not being able to make a prediction (as the mean is the default prediction in this case)

Motivated by this success, let's try out another variant which tries to improve on our baseline model introduced earlier instead of the mean rating of each user.

In [ ]:
knn_baseline = KNNBaseline(verbose = False)

knn_baseline.fit(trainset)

test_preds = knn_baseline.test(testset)
train_preds = knn_baseline.test(trainset.build_testset())

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
full_knn_matrix = get_full_user_item_matrix(knn_baseline, list(range(943)), list(range(1541))) # this can take a while

In [ ]:
show_matrix(np.clip(full_knn_matrix, 1, 5), "User-movie-matrix as predicted by the kNN model which corrects the baseline. \n (Predictions are clipped to [1, 5])")

In [ ]:
show_rating_distribution(full_knn_matrix, model = knn_baseline)

Visually the reconstructed user-movie matrix makes a quite similar impression, but we were able to push the metrics on the test set by quite a bit (Recall, for the R² score higher is better). Now we are going to explore another family of models which are very important in the field of collaborative filtering - matrix factorization based models.

## Matrix factorization

We already observed that the columns and the rows in the user-movie matrix $M$ are likely to be correlated. In more mathematical terms, this means that this matrix probably does not have _full rank_. Algorithms based on matrix factorization exploit this fact by _factoring_ this matrix as a product $A \in \mathbb{R}^{943 \times k}$ and $B \in \mathbb{R}^{k \times 1541}$ such that $M \approx AB$.   
$k$ is a free parameter here and is typically chosen much smaller than the dimensions of the user movie matrix. By construction, the rank of the factorized matrix will be at most $k$.

As if this wasn't motivation enough, another thing that we get for free from following this approach are _embeddings_ - a very fundamental concept in data science and machine learning.

The matrices $A$ and $B$ can be understood as collections of vectors of length $k$ - embeddings of the users and items in a joint vector space $\mathbb{R}^k$. By construction the interaction between user $i$ and item $j$ is then modeled as the dot product of their respective embedding vectors - it holds that $M_{ij} = A_i \cdot B_j$. 

In the literature and the surprise library this approach is catalogued under the name **SVD** (singular value decomposition), as it is based on this concept from linear algebra. Let's try it out!

### SVD Algorithm


In [ ]:
svd = SVD(n_factors=100, 
          biased=True, 
          random_state=3)

svd.fit(trainset)

In [ ]:
test_preds = svd.test(testset)
train_preds = svd.test(trainset.build_testset())

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
get_metrics_from_surprise(train_preds)

Let's have a look at the full user-movie matrix again. Note that the predictions may lie outside of the range [1, 5] again so we clip them appropriately.

In [ ]:
full_svd_matrix = get_full_user_item_matrix(svd, list(range(943)), list(range(1541))) 

In [ ]:
show_matrix(np.clip(full_svd_matrix, 1, 5), "User-movie-matrix as predicted by the SVD model. \n (Predictions are clipped to [1, 5])")

In [ ]:
show_rating_distribution(full_svd_matrix, model = svd)

In [ ]:
show_matrix(full_svd_matrix[:30, :30])


Notice how in our predictions there are movies that have a consistent scoring between different users, as well as some users that give consistent good (or bad scores) across different users. That's a pattern we were able to observe in the sparse training set as well and it also makes sense that some movies are just good and some people just like all kinds of movies. :)

### The influence of the embedding dimension

In our matrix factorization, we chose k = 100, so we assumed that the user-movie matrix has (at most) a [_rank_](https://en.wikipedia.org/wiki/Rank_(linear_algebra)) of 100. A lower rank means that the rows and columns are more correlated, meaning that there is less variance between the ratings of the users or movies, respectively. Let's see how our metrics perform if we chosse different values of k.

In [ ]:
def get_metrics_for_fixed_k(k: int, reg_lambda = 0.01):

    """
    Calculates train and test RMSE for a fixed k parameter in the SVD algorithm.
    """

    svd = SVD(n_factors=k, random_state=3, reg_all = reg_lambda, biased = False)

    svd.fit(trainset)


    train_preds = svd.test(trainset.build_testset())
    test_preds = svd.test(testset)

    train_mae, train_rmse, train_r2 = get_metrics_from_surprise(train_preds, print_ = False)
    test_mae, test_rmse, test_r2 = get_metrics_from_surprise(test_preds, print_ = False)


    return train_rmse, test_rmse, train_mae, test_mae


In [ ]:
train_rmses = []
test_rmses = []
train_maes = []
test_maes = []

k_range = np.arange(5, 120, 5)

for k in tqdm(k_range):

    train_rmse, test_rmse, train_mae, test_mae = get_metrics_for_fixed_k(k)
    train_rmses.append(train_rmse)
    test_rmses.append(test_rmse)
    train_maes.append(train_mae)
    test_maes.append(test_mae)



In [ ]:
plt.figure(figsize = (12, 8))

plt.plot(k_range, train_rmses, label = "Train Set", c = "#34B3B6", marker = "x")
plt.plot(k_range, test_rmses, label = "Test Set", c = "#8534B6", marker = "x")
plt.xlabel("k")
plt.ylabel("RMSE")
plt.title("RMSE over train and test set as a function of k")
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize = (12, 8))

plt.plot(k_range, train_maes, label = "Train Set", c = "#34B3B6", marker = "x")
plt.plot(k_range, test_maes, label = "Test Set", c = "#8534B6", marker = "x")
plt.xlabel("k")
plt.ylabel("RMSE")
#plt.ylim(.5, 1)
plt.title("MAE over train and test set as a function of k")
plt.grid()

plt.show()

In [ ]:
pd.DataFrame(list(zip(k_range, test_rmses, test_maes)), 
             columns = ["embedding dim", 
                        "test_rmse", 
                        "test_mae"]).head(8)

By choosing a larger $k$, we make the algorithm more powerful, hence it is able to achieve a lower RMSE on the training set. But the error on the test set is hardly affected by this - it is actually the lowest for k = 30 and we can achieve competitive results with k as low as 15! So it looks like the choice of $k = 100$ was way too large. 

Lets try it again with a smaller k:

In [ ]:
svd_small = SVD(n_factors=15, 
          biased=True, 
          random_state=3,
          reg_all = 0.01)

svd_small.fit(trainset)

In [ ]:
test_preds = svd_small.test(testset)
train_preds = svd_small.test(trainset.build_testset())

In [ ]:
get_metrics_from_surprise(test_preds)

In [ ]:
get_metrics_from_surprise(train_preds)

In [ ]:
full_small_svd_matrix = get_full_user_item_matrix(svd_small, list(range(943)), list(range(1541)))

In [ ]:
show_matrix(np.clip(full_small_svd_matrix, 1, 5), "User movie matrix as predicted by the SVD model with k = 15 \n Predictions are clipped to the range [1,5]")

In [ ]:
show_rating_distribution(full_small_svd_matrix, model = svd_small)

The algorithm the we used now, corresponds to a technique known as _probabilistic matrix factorization_. We just (stochastically) factorize the user-movie matrix and take the entries of result as predictions. The surprise library also offers variants of the SVD algorithm, which tries to improve on an existing baseline model. However, since the results were not significantly better I will not include them in this, already quite long, blogpost.  

## Turning these algorithms into actual recommenders

I would like to briefly illustrate how we would now turn any of these algorithms into an actual recommendation system.
In this purely collaborative setting, where we don't have further information about the users and the items, this will probably not be too enlightening but it is still a necessary building block. 

We have to do 2 things:
* For a given user, filter out the movies he has already rated
* sort the predicted ratings descendingly and return the top n (say, n = 5) movies.

Let's write a little class which does just that.

In [ ]:
class Recommender:

    def __init__(self, model):

        self.model = model
        self.ratings = pd.DataFrame([r for r in model.trainset.all_ratings()],
                                    columns = ["user_id", "movie_id", "rating"])

    def get_all_predictions_for_user(self, user):

        return np.array([get_estimate(self.model, user, j) for j in range(1682)]) 

    def get_known_movies(self, user):

        return self.ratings[self.ratings.user_id == user].movie_id.unique()


    def get_top_recommendations(self, user, top_n = 5):

        """
        For the given user, returns top_n movie_ids for which the 
        predicted ratings are the highest. Only recommends movies
        which the user has not seen yet.
        """
        
        # get the predicted ratings for every item
        predicted_ratings = self.get_all_predictions_for_user(user)

        # get the movies the user knows already
        known_movies = self.get_known_movies(user)

        #mask them in the predictions array so they don't get recommended
        masked_ratings = predicted_ratings.copy()
        masked_ratings[known_movies] = -np.inf

        return (-masked_ratings).argsort()[:top_n]

And try it out!

In [ ]:
rec = Recommender(svd_small)

In [ ]:
for user_id in np.random.randint(low = 0, high = 943, size = 3):

    print(f"Recommended movies for user {user_id}:")
    print(rec.get_top_recommendations(user = user_id))

Some movies appear in all 3 selected recommendations, some in 2 of them. This could be a coincidence, but let's check how they are rated in the training set.

In [ ]:
for idx in [52, 233, 269, 603]:
    print(f"Movie id    : {idx}")
    print(f"# ratings   : {len(train_df[train_df.movie_id == idx].rating)}")
    print(f"Mean rating : {train_df[train_df.movie_id == idx].rating.mean():.03f}\n")

So, these just seem to be popular movies which were rated by quite some people. The mean rating of movie 233 is not too high, however a look at the histogram below reveals that it is generally a well rated in the training set.

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(train_df[train_df.movie_id == 233].rating, color = "#34B3B6")
plt.title("Ratings for movie 233 in the Training Set")
plt.show()

To get a feeling for how diverse the recommendations are, let's compute the top 5 recommendations for each user and analyze them.

In [ ]:
top_recs = np.array([])

for user_id in tqdm(range(943)):
    top_recs = np.concatenate((top_recs, rec.get_top_recommendations(user = user_id, top_n = 5)))

In [ ]:
len(np.unique(top_recs))

In total, there are only 72 movies (out of the 1541 total movies) which get recommended. Let's explore how often each one of those is in the top 5 recommendations for some user.

In [ ]:
ct = Counter(top_recs.astype(int))
recommendations = ct.keys()
frequencies = ct.values()
sorted_recs, sorted_freqs = list(zip(*sorted(zip(recommendations, frequencies), key = lambda x: x[1])))

plt.figure(figsize = (24, 8))
bars = plt.bar(list(range(len(sorted_freqs))), sorted_freqs, color = "#34B3B6")
plt.title("Movies that get recommended using this system")
plt.xticks(list(range(len(sorted_freqs))), [str(num) for num in sorted_recs], rotation = 90)
#plt.bar_label(bars, padding = 3)
plt.ylabel("Number of times movie was in top 5 recommendations")
plt.xlabel("Movie id")
plt.show()



So apparently our recommendation system happily recommends the movies with ids 233 and 603 to almost every user. As these are probably very popular movies, this makes sense. 

What's cool, is that there are also quite some movies which get recommended between 10-50 times. What I also find interesting are the movies that only get recommended one or 2 times. It would be really interesting to explore if they make "sense", i.e. if they are in some sense similar to the movies a given user has rated well. But as we limit ourselves to purely collaborative filtering here, this is not possible. 

## A different evaluation strategy

So far we have framed the problem as a _regression_ problem. Also, the metrics we used for evaluation are very common in a regression setting. This makes a lot of sense for at least 2 reasons:
* The predicted variable has a clear interpretation. A higher rating simlpy means that the user is more likely to enjoy the movie more.
* When turning these algortihms into actual recommendation engines, recommendations naturally come with a _ranking_ - there can even be items rated higher than the actually possible maximal rating, as we have seen.

However, it is still interesting to evaluate the performance on our test set _as if it was_ a classification problem. We simply round the predictions to the next natural number and then take a look at the _confusion matrix_.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

def evaluate_clf_problem(df, model):

    labels = df.rating
    cont_preds = df[["user_id", "movie_id"]].apply(lambda x: model.predict(uid = x[0], iid = x[1]).est, axis = 1)
    binned_preds = bin_to_integer_ratings(cont_preds)

    cm = confusion_matrix(labels, binned_preds)

    print(classification_report(labels, binned_preds))

    return cm


In [ ]:
cm = evaluate_clf_problem(test_df, svd)

plt.figure(figsize = (15, 15))
disp = ConfusionMatrixDisplay(cm, display_labels = list(range(1, 6)))
disp.plot(ax = plt.gca())
plt.show()

A few interesting things can be seen from this confusion matrix. First, our model has made all kinds of errors on the test set, except for predicting a 1 when the real label was a 5. That is quite good news, as this is the worst kind of mistake (together with predicting a 5 when the ground truth was a 1, which happened 10 times). The most frequent type of errors are confusions between 3's and 4's and 4's and 5's, respectively. These errors are not so bad, as they still reflect the general preferences.

## Wrap up

Before we conclude this article, let's compare the different algorithms in terms of the RMSE (root mean square error).

| Query contents      | RMSE (Test Set)                 | MAE (Test Set)     |
| :---                |    :----:                       |          ---:      |
| Movie Mean             |  1.02 |  0.82 |
| User Mean              |  1.04 |  0.84 |
| Baseline Model         |  0.94 |  0.75 |
| kNN                    |  0.97 |  0.77 |
| kNN with Means         |  0.95 |  0.75 |
| kNN with Baseline      |  **0.93** |  **0.74** |
| SVD (embedding dim 100)|  0.94 |  0.75 |
| SVD (embedding dim 10) |  0.94 |  0.74 |


What I find particularly interesting, is that the baseline model with only **one** learnable parameter per user and movie (so 943 + 1541 = 2484 parameters in total) is actually better than some of the kNN based methods and also very close to the performance of the actual "best" model. This "best" model (out of the models we considered, on this specific test set) is the kNN based model which aims to improve on the baseline. I'm writing best in quotation marks here, because it is very hard to measure the success of a recommendation system with an offline metric. Offline in this case means, that the evaluation is carried out on historical data instead of in a live setting.

Usually, recommendation systems in production are hybrid models, which means that they are an ensemble of many different models. To compare different ensembles, companies use A/B testing: In a Live Setting, some user get version A and some version B. By comparing certain business metrics like click rates or conversions, it can be measures which system is better.


### Conclusion

So, to conclude this blogpost, let's summarise the main points.

We have
* Introduced the concept of collaborative filtering
* Explored some purely collaborative approches on the MovieLens100k dataset, in particular
  * a baseline model
  * some k-nearest neighbour (kNN) based models
  * an algorithm based on matrix factorization
* converted one of these algorithms into a real recommendation system and explored its recommendations
* compared the different models in terms of some offline metrics
* and discussed how evaluation of recommendation systems is a tricky thing.

I hope this blogpost has given you some insights into the inner workings of collaborative filtering algorithms. In the next article we will explore the concept of *content-based* filtering. Thanks for reading!
